In [75]:
import requests
strDelimiter='[]'

In [6]:
'''Question 1
Write a program that retrieves data representing all, what we'll call "subway" routes: "Light Rail" (type 0) and
“Heavy Rail” (type 1). The program should list their “long names” on the console.
Partial example of long name output: Red Line, Blue Line, Orange Line...
There are two ways to filter results for subway-only routes. Think about the two options below and choose:
1. Download all results from https://api-v3.mbta.com/routes then filter locally
2. Rely on the server API (i.e., https://api-v3.mbta.com/routes?filter[type]=0,1) to filter before results
are received
Please document your decision and your reasons for it.'''

#Grabbing only required data from the API, no need to get all data from MBTA API, 
#Server side filtering is already taken care of so I can utilize that to save time and resources.
def getRailResponseJSON():
    railResponse = requests.get('https://api-v3.mbta.com/routes?filter[type]=0,1')
    return railResponse.json()

In [7]:
#extracting only required information from the JSON response
def getLongName(railData):
    return [data['attributes']['long_name'] for data in railData['data']]

In [191]:
#Question 1 Answer Code block
#Calls the above functions and print the long names of the rail routes
railData = getRailResponseJSON()
longName = getLongName(railData)
print(longName)

In [9]:
'''Question 2
Extend your program so it displays the following additional information.
1. The name of the subway route with the most stops as well as a count of its stops.
2. The name of the subway route with the fewest stops as well as a count of its stops.
3. A list of the stops that connect two or more subway routes along with the relevant route names for
each of those stops.'''
# Grabbing relevant stop information from the API, and filtering it locally to get the required information
def getRouteStops(railData):
    routeId = [data['id'] for data in railData['data']]
    subwayStops = set()
    routeStops = {}
    for id in routeId:
        stopResponse = requests.get('https://api-v3.mbta.com/stops?filter[route]=' + id)
        stopData = stopResponse.json()
        stopName = [data['attributes']['name'] for data in stopData['data']]
        subwayStops.update(stopName)
        routeStops[id] = stopName
    return routeStops, subwayStops

In [10]:
#Function to only get maximun stops on a route
def getMaxStops(routeStops):
    return max(routeStops, key=lambda k: len(routeStops[k]))

In [11]:
#Function to only get minimum stops on a route
def getMinStops(routeStops):
    return min(routeStops, key=lambda k: len(routeStops[k]))

In [13]:
#Function to get the stops that connect two or more subway routes, and filtering data to be used for Question 3
def getStopRoutesAndIntersections(routeStops, subwayStops):
    stopRoutes = {}
    intersectionRouteStops = {}

    for stop in subwayStops:
        counter = 0
        routeName = ''
        for key in routeStops:
            if stop in routeStops[key]:
                counter += 1
                routeName += key + ', '
                if stop not in stopRoutes:
                    stopRoutes[stop] = [key]
                    
                else:
                    stopRoutes[stop] = stopRoutes[stop] + [key]                    

        if counter > 1:
            print(f"Stop: {stop}, Connecting Routes: {routeName[:-2]}")
        
            listOfIntersectingStops = routeName[:-2].split(', ')
            
            for intersectingStop in listOfIntersectingStops:
                if intersectingStop not in intersectionRouteStops:
                    intersectionRouteStops[intersectingStop] = [stop]
                else:
                    intersectionRouteStops[intersectingStop] = intersectionRouteStops[intersectingStop] + [stop]
    return stopRoutes, intersectionRouteStops

In [190]:
#Question 2 Answer Code block
#Calls the above functions and print the required information
routeStops, subwayStops = getRouteStops(railData)
maxStops = getMaxStops(routeStops)
minStops = getMinStops(routeStops)
print(f"Route with most stops: {maxStops}, Number of stops: {len(routeStops[maxStops])}")
print(f"Route with fewest stops: {minStops}, Number of stops: {len(routeStops[minStops])}")
stopRoutes, intersectionRouteStops = getStopRoutesAndIntersections(routeStops, subwayStops)

In [15]:
'''Question 3
Extend your program again such that the user can provide any two stops on the subway routes you listed for
question 1.
List a rail route you could travel to get from one stop to the other. We will not evaluate your solution based
upon the efficiency or cleverness of your route-finding solution. Pick a simple solution that answers the
question. We will want you to understand and be able to explain how your algorithm performs.
Examples:
1. Davis to Kendall/MIT -> Red Line
2. Ashmont to Arlington -> Red Line, Green Line B
Hint: It might be tempting to hardcode things in your algorithm that are specific to the MBTA system, but we
believe it will make things easier for you to generalize your solution so that it could work for different and/or
larger subway systems.
How you handle input, represent train routes, and present output is your choice.'''

#startstop and endstop
#1) find route(s) startstop is on
#2) check if endstop is on the same route, return route
#3) if not, take list of route(s) startstop is on, traverse to next intersection point on route
#4) update startstop to intersectionstop, record new route, repeat 1-3

def findRoute(start, end, routeStops, stopRoutes, intersectionRouteStops,traversedRoutes = [], traversedIntersectionStops = []):
    routesFromStart = stopRoutes[start]
    #print(f"start: {start}, end: {end}")
    
    if start not in traversedIntersectionStops:
        traversedIntersectionStops.append(start)

    # if the start point is an intersection of multiple routes      
    for route in routesFromStart:
        if route not in traversedRoutes:
            
            #get list of all stops in the specified route
            stopList = routeStops[route] 
            #if endpoint found then return all routes traversed from start to end
            if end in stopList:  
                traversedRoutes.append(route)
                return traversedRoutes
            #if endpoint not found, get list of all intersecting points of the route, and traverse through them
            else: 
                #add starting route to traversed routes
                traversedRoutes.append(route) 
                intersectionStops = intersectionRouteStops[route]
                for intersectingStop in intersectionStops:
                    if intersectingStop in traversedIntersectionStops:
                        continue
                    #print(f"start: {start}, intersectingStop: {intersectingStop}")
                    if intersectingStop is start:
                        continue
                    traversedIntersectionStops.append(intersectingStop)
                    result = findRoute(intersectingStop, end, routeStops, stopRoutes, intersectionRouteStops, traversedRoutes, traversedIntersectionStops)
                    if result is not None:
                        return result
                    
    return traversedRoutes

In [167]:
#Function to find the shortest traversable path between two stops
#I was getting unexpected results when I was trying to keep track of traversed routes and intersection stops, so I switched to using strings to keep track of traversal
#this way instead of a pointer being recursively passed through the function that would be updated by the recursive calls at unexpected times, the string can be updated and passed through the function without any issues
#Mostly the same logic as the FindRoute function, but with the addition of a list to keep track of all valid route paths, and returning the shortest path
def findShortestRoute(start, end, routeStops, stopRoutes, intersectionRouteStops,traversedRoutes = "", traversedIntersectionStops = ""):
    
    routesFromStart = stopRoutes[start]
    #print(f"start: {start}, end: {end}")
    listOfValidRoutePaths = [] 
    validRoutePath = []
    
    if start not in traversedIntersectionStops:
        traversedIntersectionStops=traversedIntersectionStops+start+strDelimiter#'[]'

    # if the start point is an intersection of multiple routes      
    for route in routesFromStart:

        if route not in traversedRoutes.split('[]'):#traversedRoutes:
            
            #get list of all stops in the specified route
            stopList = routeStops[route] 

            #if endpoint found then return all routes traversed from start to end
            if end in stopList:  
                traversedRoutes=traversedRoutes+route+strDelimiter#'[]'
                return traversedRoutes
            #if endpoint not found, get list of all intersecting points of the route, and traverse through them
            else: 
                #add starting route to traversed routes
                traversedRoutes=traversedRoutes+route+strDelimiter#'[]'

                intersectionStops = intersectionRouteStops[route]
                for intersectingStop in intersectionStops:
                    if intersectingStop in traversedIntersectionStops.split('[]'):#traversedIntersectionStops
                        continue

                    if intersectingStop is start:
                        continue
                    traversedIntersectionStops = traversedIntersectionStops+intersectingStop+strDelimiter #.append(intersectingStop)
                    validRoutePath = findShortestRoute(intersectingStop, end, routeStops, stopRoutes, intersectionRouteStops, traversedRoutes, traversedIntersectionStops)
                    if (validRoutePath is not None): 
                        listOfValidRoutePaths.append(validRoutePath) 

                #check if listOfValidRoutePaths is empty, if not, return the shortest path
                if listOfValidRoutePaths:
                    #I count the number of times the global strDelimiter appears because there might be an instance where a route has an extremely long name
                    #The route with the least strDelimiter count is the shortest route as the strDelimiter is only added when a new route is traversed
                    minimumValidRoute = (min(listOfValidRoutePaths, key=lambda x: x.count(strDelimiter)))
                    return minimumValidRoute

                else:
                    return None

In [184]:
def inputStops(subwayStops, start, end):
    while True:
        #print(f"Start: {start}, End: {end}")
        if start == "":
            start = input("Enter start stop or 'quit': ").rstrip()
            
        if start == 'quit':
            return "",""
        elif start not in subwayStops :
            print("The Starting Station is invalid. Please re-enter a correct station.")
            start = ''
            continue

        if end == "":
            end = input("Enter end stop or 'quit': ").rstrip()
            
        if end == 'quit':
            return "",""
        elif end not in subwayStops:
            print("The Ending Station is invalid. Please re-enter a correct station.")
            end = ''
        elif start == end:
            print("Start and end stations cannot be the same. Please re-enter two different stations.")
            start = ''
            end = ''
        
        if start in subwayStops and end in subwayStops:
            break
    return start, end

In [189]:
#Question 3 Answer Code block
#Calls the above methods to get stop inputs and find a route between the two stops and the shortest route between the two stops
start, end = inputStops(subwayStops, "", "") #'Alewife', 'Airport' #'Ashmont', 'Arlington'#inputStops(subwayStops, "", "") #'Alewife', 'Airport' 
#print(f"Start: {start}, End: {end}")
if start != "" or end != "":
    listOfRoutes = findRoute(start, end, routeStops, stopRoutes, intersectionRouteStops, [], [])
    stringOfRoutes = ','.join(map(str, listOfRoutes)) 
    print(f"A Route from {start} to {end}: {stringOfRoutes}")

    listOfShortestRoute = findShortestRoute(start, end, routeStops, stopRoutes, intersectionRouteStops, "", "")
    convertListOfMinimumRoute=listOfShortestRoute[:-2].split('[]')
    #print(convertListOfMinimumRoute)
    stringOfMinimumRoute = ','.join(map(str, convertListOfMinimumRoute))
    print(f"The Shortest Route from {start} to {end}: {stringOfMinimumRoute}")
     